In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import *
from pyspark.sql.window import Window
# Initialize Spark Session
spark = SparkSession.builder.appName("BMS").getOrCreate()

# Create Schema and Data
data = [
    (1, 'N'), (2, 'Y'), (3, 'N'), (4, 'Y'), (5, 'Y'),
    (6, 'Y'), (7, 'N'), (8, 'Y'), (9, 'Y'), (10, 'Y'),
    (11, 'Y'), (12, 'N'), (13, 'Y'), (14, 'Y')
]

columns = ["seat_no", "is_empty"]

# Create DataFrame
bms_df = spark.createDataFrame(data, columns)

# Show Data
bms_df.createOrReplaceTempView("bms")


In [8]:
spark.sql(
"""
    with seats as (
    select 
    seat_no,
    is_empty,
    lag(is_empty, 1) over(order by seat_no) as prev_1,
    lag(is_empty, 2) over(order by seat_no) as prev_2,
    lead(is_empty, 1) over(order by seat_no) as next_1,
    lag(is_empty, 2) over(order by seat_no) as next_2
    from bms)
    
    select * from seats where
    (is_empty = 'Y' and prev_1 = 'Y' and prev_2 = 'Y') or 
    (is_empty = 'Y' and prev_1 = 'Y' and prev_2 = 'Y') or 
    (is_empty = 'Y' and prev_1 = 'Y' and prev_2 = 'Y')
    
    
""").show()

+-------+--------+------+------+------+------+
|seat_no|is_empty|prev_1|prev_1|prev_1|prev_2|
+-------+--------+------+------+------+------+
|      1|       N|  null|  null|     Y|  null|
|      2|       Y|     N|  null|     N|  null|
|      3|       N|     Y|     N|     Y|     N|
|      4|       Y|     N|     Y|     Y|     Y|
|      5|       Y|     Y|     N|     Y|     N|
|      6|       Y|     Y|     Y|     N|     Y|
|      7|       N|     Y|     Y|     Y|     Y|
|      8|       Y|     N|     Y|     Y|     Y|
|      9|       Y|     Y|     N|     Y|     N|
|     10|       Y|     Y|     Y|     Y|     Y|
|     11|       Y|     Y|     Y|     N|     Y|
|     12|       N|     Y|     Y|     Y|     Y|
|     13|       Y|     N|     Y|     Y|     Y|
|     14|       Y|     Y|     N|  null|     N|
+-------+--------+------+------+------+------+

